# Lab 2 - Unsupervised Feature Learning with PyTorch

## Objective

* To perform unsupervised feature learning via autoencoder in PyTorch.

**Suggested reading**: 
* [Autoencoder - Wikipedia](https://en.wikipedia.org/wiki/Autoencoder)
* [Autograd tutorial](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)

## Why

[Unsupervised learning](https://en.wikipedia.org/wiki/Unsupervised_learning) is a fundamental problem in machine learning where we aim to learn from unlabelled data. Unsupervised feature learning or representation learning can find wide usage in various applications for extracting useful information from often abundant unlabelled data.

## 1. Autograd: Automatic Differentiation

In the previous lab, we briefly covered **Tensor** and **Computational Graph**. We have actually used **Autograd** already. Here, we learn the basics below, a condensed and modified version of the original [PyTorch tutorial on Autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)

#### Why differentiation is important? 

This is because it is a key procedure in **optimisation** to find the optimial solution of a loss function. The process of learning/training aims to minimise a predefined loss.

#### How automatic differentiation is done in pytorch?
The PyTorch ``autograd`` package makes differentiation (almost) transparent to you by providing automatic differentiation for all operations on Tensors, unless you donot want it (to save time and space). 

A ``torch.Tensor`` type variable has an attribute ``.requires_grad``. Setting this attribute ``True`` tracks (but not computes yet) all operations on it. After we define the forward pass, and hence the *computational graph*, we call ``.backward()`` and all the gradients will be computed automatically and accumulated into the ``.grad`` attribute. 

This is made possible by the [**chain rule of differentiation**](https://en.wikipedia.org/wiki/Chain_rule).

#### How to stop automatic differentiation (e.g., because it is not needed)
Calling method ``.detach()`` of a tensor will detach it from the computation history. We can also wrap the code block in ``with torch.no_grad():`` so all tensors in the block do not track the gradients, e.g., in the test/evaluation stage. 

### Function

``Tensor``s are connected by ``Function`` to build an acyclic *computational graph* to encode a complete history of computation. The ``.grad_fn`` attribute of a tensor references a ``Function`` created
the ``Tensor``, i.e., this ``Tensor`` is the output of its ``.grad_fn`` in the computational graph.

Learn more about autograd by referring to the [documentation on autograd](https://pytorch.org/docs/stable/autograd.html)

## 2. Building and Optimising an Autoencoder in Pytorch

We focus on *computer vision* applications. We are going to build an autoencoder to learn a low-dimensional representation of some specific images, in a particular dataset. We just proceed and explain as we go. This part follows the [Autoencoder notebook by Lisa Zhang](https://www.cs.toronto.edu/~lczhang/aps360_20191/lec/w05/autoencoder.html) with additional explanation and minor changes.

#### Libaries

Get ready by importing the standard APIs

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

#### Data
Let us work with the popular [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) with handwritten digits. We will work on a subset for efficiency.

In [ ]:
mnist_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
print(len(mnist_data))
mnist_data = list(mnist_data)[:4096]
print(len(mnist_data))

#### Define the NN architecture
In Lab 1, we did not define a class for our linear regression NN. Here we do so and define an autoencoder class consisting of an **encoder** followed by a **decoder** as defined below.
<img src="https://miro.medium.com/max/3524/1*oUbsOnYKX5DEpMOK3pH_lg.png" style="width:360px;"/>

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # 1 input image channel, 16 output channel, 3x3 square convolution
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  #to range [0, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

`__init__()` defines the layers.  `forward()` defines the *forward pass* that transform the input to the output. `backward()` is automatically defined using `autograd`.

Here, we have both convolution layers `Conv2d()` and transpose convolution layers `ConvTranspose2d()`, with nice illustrations at [Convolution arithmetic](https://github.com/vdumoulin/conv_arithmetic). The basic ones are reproduced below where blue maps indicate inputs, and cyan maps indicate outputs.

<table>
    <tr>
    <td  style="text-align: left"> Convolution with no padding, no strides.      <img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_no_strides.gif" alt="Drawing" style="width: 250px;"/> </td>
    <td  style="text-align: left"> Transpose convolution with No padding, no strides.<img src="https://github.com/vdumoulin/conv_arithmetic/raw/master/gif/no_padding_no_strides_transposed.gif" alt="Drawing" style="width: 250px;"/> </td>
</tr>
</table>

`ReLu()` and `Sigmoid()` are [rectified linear unit (ReLU)](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) and [Sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function), two popular **activation function** that performs a *nonlinear* transformation/mapping of an input variable (element-wise operation).


#### Inspect the NN architecture

Now let's take a look at the autoencoder built.

In [ ]:
myAE=Autoencoder()
print(myAE)

Let us check the (randomly initialised) parameters of this NN. Below, we check the first 2D convolution and the ReLu activiation function. 

In [ ]:
params = list(myAE.parameters())
print(len(params))
print(params[0].size())  # First Conv2d's .weight
print(params[1].size())  # First Conv2d's .bias
print(params[1])

To learn more about these functions, refer to the [`torch.nn` documentation](https://pytorch.org/docs/stable/nn.html) (search for the function, e.g., search for `torch.nn.ReLu(` and you will find its documentation [here](https://pytorch.org/docs/stable/nn.html?highlight=relu#torch.nn.ReLU)

#### Train the NN
Next, we will feed data in this autoencoder to train it, i.e., learn its parameters so that the reconstruction error (the `loss`) is minimised, using the mean square error (MSE) and `Adam` optimiser. The dataset is loaded in batches to train the model. One `epoch` means one cycle through the full training dataset. The `outputs` at the end of each epoch save the orignal image and the reconstructed (decoded) image pairs for later inspection. The steps are 
* Define the optimisation criteria and optimisation method.
* Iterate through the whole dataset in batches, for a number of `epochs` till a maximum specified or a convergence criteria (e.g., successive change of loss < 0.000001)
* In each batch processing, we 
    * do a forward pass
    * compute the loss
    * backpropagate the loss via `autograd`
    * update the parameters

In [ ]:
#Hyperparameters for training
batch_size=64
learning_rate=1e-3
max_epochs = 20

#Set the random seed for reproducibility 
torch.manual_seed(509) 
#Choose mean square error loss
criterion = nn.MSELoss() 
#Choose the Adam optimiser
optimizer = torch.optim.Adam(myAE.parameters(), lr=learning_rate, weight_decay=1e-5)
#Specify how the data will be loaded in batches (with random shffling)
train_loader = torch.utils.data.DataLoader(mnist_data, batch_size=batch_size, shuffle=True)
#Storage
outputs = []

#Start training
for epoch in range(max_epochs):
    for data in train_loader:
        img, label = data
        optimizer.zero_grad()
        recon = myAE(img)
        loss = criterion(recon, img)
        loss.backward()
        optimizer.step()            
    if (epoch % 3) == 0:
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
    outputs.append((epoch, img, recon),)

Take a look at how `autograd` keeps track of the gradients for back propagation.

In [ ]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])

We can see the loss is being well minimised. We can inspect the reconstructed images at different epochs below. Note here we use `.detach()` because the gradients are not needed for inspection purpose here. 

In [ ]:
numImgs=12;
for k in range(0, max_epochs, 9):
    plt.figure(figsize=(numImgs, 2))
    imgs = outputs[k][1].detach().numpy()    
    recon = outputs[k][2].detach().numpy()
    for i, item in enumerate(imgs):
        if i >= numImgs: break
        plt.subplot(2, numImgs, i+1)
        plt.imshow(item[0])
        
    for i, item in enumerate(recon):
        if i >= numImgs: break
        plt.subplot(2, numImgs, numImgs+i+1)
        plt.imshow(item[0])

#### Generate synthesised images
We can **interpolate** between two images via the learned embeddding. Let us pick the second `7` and four image `3` from the first epoch and obtain the low-dimensional embedding using the learned encoder.

In [ ]:
epochIndex=0;
img1Index=1;
img2Index=3;

imgs = outputs[epochIndex][1].detach().numpy()
x1 = outputs[epochIndex][1][img1Index,:,:,:];# first image
x2 = outputs[epochIndex][1][img2Index,:,:,:] # second image
x = torch.stack([x1,x2])     # stack them together so we only call `encoder` once
embedding = myAE.encoder(x)
e1 = embedding[0] # embedding of first image
e2 = embedding[1] # embedding of second image
print(e1.size())

In the embedding space, we do a linear interpolation between the two embeddings and then decode these interpolated embeddings into images.

In [ ]:
embedding_values = []
for i in range(0, 10):
    e = e1 * (i/10) + e2 * (10-i)/10
    embedding_values.append(e)
embedding_values = torch.stack(embedding_values)

recons = myAE.decoder(embedding_values)
plt.figure(figsize=(10, 2))
for i, recon in enumerate(recons.detach().numpy()):
    plt.subplot(2,10,i+1)
    plt.imshow(recon[0])
plt.subplot(2,10,11)
plt.imshow(imgs[img2Index][0])
plt.subplot(2,10,20)
plt.imshow(imgs[img1Index][0])

## 3. Exercises


* Code PCA using ``torch.nn`` and compare it with the close-form solution via eigendecomposition.
* Try out different optimisers or different loss function (the L1loss, MAE) and compare the results.
* Change the architecture of autoencoder (e.g., depth, other layers such as max pooling, different activation functions) to compare the results.
* Repeat the above on a subset from the CIFAR10 dataset. See example usage [here](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html). For example, you can interpolate a cat and a dog.



# Acknowledgement
Some part of this notebook is adapted from the following sources

* [Autoencoder notebook by Lisa Zhang from APS360, University of Toronto](https://www.cs.toronto.edu/~lczhang/aps360_20191/lec/w05/autoencoder.html)

# References

* [CS231n: Convolutional Neural Networks for Visual Recognition from Stanford](http://cs231n.github.io/)